<a href="https://colab.research.google.com/github/b-siddharth/_ML/blob/main/ann.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing the Libraries

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [2]:
tf.__version__

'2.9.2'

**Data Preprocessing**

Importing the dataset

In [4]:
data = pd.read_csv('Churn_Modelling.csv')
x = data.iloc[:, 3:-1].values
y = data.iloc[:,-1].values

In [5]:
print(x)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


Encoding the data

In [9]:
from sklearn.preprocessing import LabelEncoder
encode = LabelEncoder()
x[:, 2] = encode.fit_transform(x[:, 2]) #gender column

In [10]:
print(x)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


In [11]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers = [('encoder', OneHotEncoder(), [1])], remainder='passthrough')
x = np.array(ct.fit_transform(x))

In [12]:
print(x)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


Spliting the dataset into training and testing

In [13]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.25, random_state=0)

Feature Scaling

In [14]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

**Building the Neural Network**

Initialization of the ANN

In [15]:
ann = tf.keras.models.Sequential()

Adding the input and hidden layers

In [16]:
ann.add(tf.keras.layers.Dense(units='6', activation='relu'))

Adding another layer 

In [17]:
ann.add(tf.keras.layers.Dense(units='6', activation='relu'))

Adding an output layer

In [18]:
ann.add(tf.keras.layers.Dense(units='1', activation='sigmoid'))

**Training the model**

Compling the ANN

In [19]:
ann.compile(optimizer='adam', loss='binary_crossentropy', metrics= ['accuracy'])

Training using the training set

In [20]:
ann.fit(x_train, y_train, batch_size=32, epochs=100)

Epoch 1/100
235/235 [==============================] - 1s 2ms/step - loss: 0.6680 - accuracy: 0.6563
Epoch 2/100
235/235 [==============================] - 0s 2ms/step - loss: 0.4976 - accuracy: 0.7956
Epoch 3/100
235/235 [==============================] - 0s 2ms/step - loss: 0.4625 - accuracy: 0.7967
Epoch 4/100
235/235 [==============================] - 0s 2ms/step - loss: 0.4436 - accuracy: 0.7985
Epoch 5/100
235/235 [==============================] - 0s 2ms/step - loss: 0.4329 - accuracy: 0.8024
Epoch 6/100
235/235 [==============================] - 0s 2ms/step - loss: 0.4271 - accuracy: 0.8037
Epoch 7/100
235/235 [==============================] - 0s 2ms/step - loss: 0.4225 - accuracy: 0.8055
Epoch 8/100
235/235 [==============================] - 0s 2ms/step - loss: 0.4189 - accuracy: 0.8063
Epoch 9/100
235/235 [==============================] - 0s 1ms/step - loss: 0.4154 - accuracy: 0.8073
Epoch 10/100
235/235 [==============================] - 0s 2ms/step - loss: 0.4111 - accura

**Predicting single observations**

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: $ 60000

Number of Products: 2

Does this customer have a credit card? Yes

Is this customer an Active Member: Yes

Estimated Salary: $ 50000

In [22]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)
#0.5 since we are using a sigmoid activation and the predicted probablity is being returned

1/1 [==============================] - 0s 75ms/step
[[False]]


So for the given conditions the customer would stay in the bank

Using test set results

In [23]:
y_pred = ann.predict(x_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

79/79 [==============================] - 0s 1ms/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


Performance Measure - using Confusion Matrix

In [24]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
print()
accuracy_score(y_test, y_pred)

[[1879  112]
 [ 246  263]]



0.8568